In [75]:
import csv
import math
import sys
from datetime import datetime as dt
from datetime import timezone as tz

import numpy as np
import pandas as pd

import utility

### clean data

In [59]:
df = pd.read_excel('../GhostPostCC/data/Region_MIDW.xlsx')

In [60]:
df

,Region,UTC time,Local date,Hour,Local time,Time zone,DF,D,NG,TI,...,CO2 Emissions: OIL,CO2 Emissions: Other,CO2 Emissions Generated,CO2 Emissions Imported,CO2 Emissions Exported,CO2 Emissions Consumed,Positive Generation,Consumed Electricity,CO2 Emissions Intensity for Generated Electricity,CO2 Emissions Intensity for Consumed Electricity
0,MIDW,2015-07-01 06:00:00,2015-07-01,2,2015-07-01 02:00:00,Eastern,76958,73432.0,70103.0,-2054.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MIDW,2015-07-01 07:00:00,2015-07-01,3,2015-07-01 03:00:00,Eastern,74126,70211.0,64905.0,-2421.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MIDW,2015-07-01 08:00:00,2015-07-01,4,2015-07-01 04:00:00,Eastern,72102,68163.0,62094.0,-2423.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MIDW,2015-07-01 09:00:00,2015-07-01,5,2015-07-01 05:00:00,Eastern,71300,67309.0,60872.0,-2375.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MIDW,2015-07-01 10:00:00,2015-07-01,6,2015-07-01 06:00:00,Eastern,72363,68456.0,61409.0,-2603.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77298,MIDW,2024-04-25 00:00:00,2024-04-24,20,2024-04-24 20:00:00,Eastern,75585,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77299,MIDW,2024-04-25 01:00:00,2024-04-24,21,2024-04-24 21:00:00,Eastern,76035,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77300,MIDW,2024-04-25 02:00:00,2024-04-24,22,2024-04-24 22:00:00,Eastern,75837,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77301,MIDW,2024-04-25 03:00:00,2024-04-24,23,2024-04-24 23:00:00,Eastern,73175,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77303 entries, 0 to 77302
Data columns (total 59 columns):
 #   Column                                             Non-Null Count  Dtype         
---  ------                                             --------------  -----         
 0   Region                                             77303 non-null  object        
 1   UTC time                                           77303 non-null  datetime64[ns]
 2   Local date                                         77303 non-null  datetime64[ns]
 3   Hour                                               77303 non-null  int64         
 4   Local time                                         77303 non-null  datetime64[ns]
 5   Time zone                                          77303 non-null  object        
 6   DF                                                 77303 non-null  int64         
 7   D                                                  77279 non-null  float64       
 8   NG              

In [62]:
df.columns

Index(['Region', 'UTC time', 'Local date', 'Hour', 'Local time', 'Time zone',
       'DF', 'D', 'NG', 'TI', 'Sum (NG)', 'NG: COL', 'NG: NG', 'NG: NUC',
       'NG: OIL', 'NG: WAT', 'NG: SUN', 'NG: WND', 'NG: OTH', 'NG: UNK',
       'Sum (Trade)', 'Sum (Imports)', 'Sum (Exports)', 'CAL', 'CAR', 'CENT',
       'FLA', 'MIDA', 'MIDW', 'NE', 'NW', 'NY', 'SE', 'SW', 'TEN', 'TEX',
       'CAN', 'MEX', 'Balance NG D TI', 'Balance TI Trade', 'Balance NG',
       'CO2 Factor: COL', 'CO2 Factor: NG', 'CO2 Factor: OIL',
       'Positive Gen from COL', 'Positive Gen from NG',
       'Positive Gen from OIL', 'CO2 Emissions: COL', 'CO2 Emissions: NG',
       'CO2 Emissions: OIL', 'CO2 Emissions: Other', 'CO2 Emissions Generated',
       'CO2 Emissions Imported', 'CO2 Emissions Exported',
       'CO2 Emissions Consumed', 'Positive Generation', 'Consumed Electricity',
       'CO2 Emissions Intensity for Generated Electricity',
       'CO2 Emissions Intensity for Consumed Electricity'],
      dtype='obj

In [76]:
cols = ['UTC time', 'Local date', 'Hour', 'Local time',
       'Sum (NG)', 'NG: COL', 'NG: NG', 'NG: NUC', 'NG: SUN',
       'NG: OIL', 'NG: WAT', 'NG: SUN', 'NG: WND', 'NG: OTH']

time_filter = (df['Local time']>'2022-01-01') & (df['Local time']<='2024-01-01')

In [115]:
data = df[time_filter][cols].reset_index(drop=True)
data

,UTC time,Local date,Hour,Local time,Sum (NG),NG: COL,NG: NG,NG: NUC,NG: SUN,NG: OIL,NG: WAT,NG: SUN,NG: WND,NG: OTH
0,2022-01-01 06:00:00,2022-01-01,1,2022-01-01 01:00:00,64543.0,25663.0,12065.0,12192.0,0.0,NaN,569.0,0.0,13584.0,470.0
1,2022-01-01 07:00:00,2022-01-01,2,2022-01-01 02:00:00,63027.0,24282.0,11574.0,12192.0,0.0,NaN,529.0,0.0,13982.0,468.0
2,2022-01-01 08:00:00,2022-01-01,3,2022-01-01 03:00:00,62214.0,23480.0,11296.0,12192.0,0.0,NaN,522.0,0.0,14256.0,468.0
3,2022-01-01 09:00:00,2022-01-01,4,2022-01-01 04:00:00,61693.0,22871.0,11026.0,12198.0,0.0,NaN,529.0,0.0,14600.0,469.0
4,2022-01-01 10:00:00,2022-01-01,5,2022-01-01 05:00:00,61166.0,22423.0,10701.0,12195.0,0.0,NaN,527.0,0.0,14850.0,470.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2024-01-01 01:00:00,2023-12-31,20,2023-12-31 20:00:00,78078.0,29329.7,30487.3,8204.0,2.0,0.0,1088.0,2.0,8596.0,371.0
17516,2024-01-01 02:00:00,2023-12-31,21,2023-12-31 21:00:00,75593.9,27283.2,30073.8,8204.0,2.9,0.0,909.0,2.9,8753.0,368.0
17517,2024-01-01 03:00:00,2023-12-31,22,2023-12-31 22:00:00,74670.1,26415.1,30143.0,8206.0,3.0,0.0,905.0,3.0,8635.0,363.0
17518,2024-01-01 04:00:00,2023-12-31,23,2023-12-31 23:00:00,73296.0,25840.3,30098.7,8203.0,3.0,0.0,878.0,3.0,7908.0,365.0


In [116]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   UTC time    17520 non-null  datetime64[ns]
 1   Local date  17520 non-null  datetime64[ns]
 2   Hour        17520 non-null  int64         
 3   Local time  17520 non-null  datetime64[ns]
 4   Sum (NG)    17520 non-null  float64       
 5   NG: COL     17520 non-null  float64       
 6   NG: NG      17520 non-null  float64       
 7   NG: NUC     17520 non-null  float64       
 8   NG: SUN     17520 non-null  float64       
 9   NG: OIL     5184 non-null   float64       
 10  NG: WAT     17520 non-null  float64       
 11  NG: SUN     17520 non-null  float64       
 12  NG: WND     17520 non-null  float64       
 13  NG: OTH     17520 non-null  float64       
dtypes: datetime64[ns](3), float64(10), int64(1)
memory usage: 1.9 MB


In [117]:
# omit NG: SUN bcs no sun data
# omit NG: OIL bcs too many nans
# change NG: OTH to become (Sum (NG) - ('NG: COL' + 'NG: NG' + 'NG: NUC' + 'NG: WAT' + 'NG: SUN' + 'NG: WND'))

data = data.drop(['NG: SUN', 'NG: OIL'], axis=1)

data['NG: OTH'] = 0
data['NG: OTH'] = data['Sum (NG)'] - data[['NG: COL', 'NG: NG', 'NG: NUC', 'NG: WAT', 'NG: WND']].sum(axis=1)
data = data.drop(['Sum (NG)', 'Local date', 'Hour'], axis=1)

In [118]:
data

,UTC time,Local time,NG: COL,NG: NG,NG: NUC,NG: WAT,NG: WND,NG: OTH
0,2022-01-01 06:00:00,2022-01-01 01:00:00,25663.0,12065.0,12192.0,569.0,13584.0,470.0
1,2022-01-01 07:00:00,2022-01-01 02:00:00,24282.0,11574.0,12192.0,529.0,13982.0,468.0
2,2022-01-01 08:00:00,2022-01-01 03:00:00,23480.0,11296.0,12192.0,522.0,14256.0,468.0
3,2022-01-01 09:00:00,2022-01-01 04:00:00,22871.0,11026.0,12198.0,529.0,14600.0,469.0
4,2022-01-01 10:00:00,2022-01-01 05:00:00,22423.0,10701.0,12195.0,527.0,14850.0,470.0
...,...,...,...,...,...,...,...,...
17515,2024-01-01 01:00:00,2023-12-31 20:00:00,29329.7,30487.3,8204.0,1088.0,8596.0,373.0
17516,2024-01-01 02:00:00,2023-12-31 21:00:00,27283.2,30073.8,8204.0,909.0,8753.0,370.9
17517,2024-01-01 03:00:00,2023-12-31 22:00:00,26415.1,30143.0,8206.0,905.0,8635.0,366.0
17518,2024-01-01 04:00:00,2023-12-31 23:00:00,25840.3,30098.7,8203.0,878.0,7908.0,368.0


In [119]:
data.to_csv('data/electricity_cleaned.csv')

In [120]:
source = 'coal'
IN_FILE_NAME = "data/electricity_cleaned.csv"

In [121]:
NUM_FEATURES_DICT = {"coal":6, "nat_gas":6, "nuclear":6, "oil":6, "hydro":11, "solar": 11,
                    "wind":11, "unknown": 6, "biomass": 6, "geothermal":6}

NUM_VAL_DAYS = 30
NUM_TEST_DAYS = 184
TRAINING_WINDOW_HOURS = 24
PREDICTION_WINDOW_HOURS = 24
MODEL_SLIDING_WINDOW_LEN = 24

COAL = 4
NAT_GAS = 5
NUCLEAR = 6
HYDRO = 7
WIND = 8
OTHERS = 9 

FUEL = {3:"coal", 4:"nat_gas", 5:"nuclear", 6:"oil", 7:"hydro", 8:"solar",
                    9:"wind", 10:"unknown"}

SOURCE_TO_SOURCE_COL_MAP = {"coal": COAL, "nat_gas" : NAT_GAS, "nuclear" : NUCLEAR, "hydro" : HYDRO, "wind" : WIND}
SOURCE_COL = SOURCE_TO_SOURCE_COL_MAP[source]
NUM_FEATURES = NUM_FEATURES_DICT[FUEL[SOURCE_COL]]

In [122]:
def initDataset(inFileName, sourceCol):
    dataset = pd.read_csv(inFileName, header=0, infer_datetime_format=True, parse_dates=['UTC time'], index_col=['UTC time'])

    print(dataset.head())
    print(dataset.columns)
    dateTime = dataset.index.values
    
    print("\nAdding features related to date & time...")
    modifiedDataset = utility.addDateTimeFeatures(dataset, dateTime, sourceCol)
    dataset = modifiedDataset
    print("Features related to date & time added")
    
    for i in range(sourceCol, len(dataset.columns.values)):
        col = dataset.columns.values[i]
        dataset[col] = dataset[col].astype(np.float64)
        # print(col, dataset[col].dtype)

    return dataset, dateTime

In [123]:
dataset, dateTime = initDataset(IN_FILE_NAME, SOURCE_COL)

/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/1023274769.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dataset = pd.read_csv(inFileName, header=0, infer_datetime_format=True, parse_dates=['UTC time'], index_col=['UTC time'])


                     Unnamed: 0           Local time  NG: COL   NG: NG  \
UTC time                                                                 
2022-01-01 06:00:00           0  2022-01-01 01:00:00  25663.0  12065.0   
2022-01-01 07:00:00           1  2022-01-01 02:00:00  24282.0  11574.0   
2022-01-01 08:00:00           2  2022-01-01 03:00:00  23480.0  11296.0   
2022-01-01 09:00:00           3  2022-01-01 04:00:00  22871.0  11026.0   
2022-01-01 10:00:00           4  2022-01-01 05:00:00  22423.0  10701.0   

                     NG: NUC  NG: WAT  NG: WND  NG: OTH  
UTC time                                                 
2022-01-01 06:00:00  12192.0    569.0  13584.0    470.0  
2022-01-01 07:00:00  12192.0    529.0  13982.0    468.0  
2022-01-01 08:00:00  12192.0    522.0  14256.0    468.0  
2022-01-01 09:00:00  12198.0    529.0  14600.0    469.0  
2022-01-01 10:00:00  12195.0    527.0  14850.0    470.0  
Index(['Unnamed: 0', 'Local time', 'NG: COL', 'NG: NG', 'NG: NUC', 'NG: WAT

In [124]:
dataset

,Unnamed: 0,Local time,NG: COL,NG: NG,NG: NUC,hour_sin,hour_cos,month_sin,month_cos,weekend,NG: WAT,NG: WND,NG: OTH
UTC time,,,,,,,,,,,,,
2022-01-01 06:00:00,0,2022-01-01 01:00:00,25663.0,12065.0,12192.0,1.000000,6.123234e-17,0.004301,0.999991,1.0,569.0,13584.0,470.0
2022-01-01 07:00:00,1,2022-01-01 02:00:00,24282.0,11574.0,12192.0,0.965926,-2.588190e-01,0.005017,0.999987,1.0,529.0,13982.0,468.0
2022-01-01 08:00:00,2,2022-01-01 03:00:00,23480.0,11296.0,12192.0,0.866025,-5.000000e-01,0.005734,0.999984,1.0,522.0,14256.0,468.0
2022-01-01 09:00:00,3,2022-01-01 04:00:00,22871.0,11026.0,12198.0,0.707107,-7.071068e-01,0.006451,0.999979,1.0,529.0,14600.0,469.0
2022-01-01 10:00:00,4,2022-01-01 05:00:00,22423.0,10701.0,12195.0,0.500000,-8.660254e-01,0.007168,0.999974,1.0,527.0,14850.0,470.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-01 01:00:00,17515,2023-12-31 20:00:00,29329.7,30487.3,8204.0,0.258819,9.659258e-01,-0.007884,0.999969,0.0,1088.0,8596.0,373.0
2024-01-01 02:00:00,17516,2023-12-31 21:00:00,27283.2,30073.8,8204.0,0.500000,8.660254e-01,-0.007168,0.999974,0.0,909.0,8753.0,370.9
2024-01-01 03:00:00,17517,2023-12-31 22:00:00,26415.1,30143.0,8206.0,0.707107,7.071068e-01,-0.006451,0.999979,0.0,905.0,8635.0,366.0


In [125]:
trainData, valData, testData, fullTrainData = utility.splitDataset(dataset.values, NUM_TEST_DAYS, NUM_VAL_DAYS)
# trainDates = dateTime[: -(NUM_TEST_DAYS*24)]
# fullTrainDates = np.copy(trainDates)
# trainDates, validationDates = trainDates[: -(NUM_VAL_DAYS*24)], trainDates[-(NUM_VAL_DAYS*24):]
# testDates = dateTime[-(NUM_TEST_DAYS*24):]
# trainData = trainData[:, SOURCE_COL: SOURCE_COL+NUM_FEATURES]
# valData = valData[:, SOURCE_COL: SOURCE_COL+NUM_FEATURES]
# testData = testData[:, SOURCE_COL: SOURCE_COL+NUM_FEATURES]

No. of rows in dataset: 17520
No. of rows in training set: 12384
No. of rows in validation set: 720
No. of rows in test set: 4416


In [126]:
trainData

array([[0, '2022-01-01 01:00:00', 25663.0, ..., 569.0, 13584.0, 470.0],
       [1, '2022-01-01 02:00:00', 24282.0, ..., 529.0, 13982.0, 468.0],
       [2, '2022-01-01 03:00:00', 23480.0, ..., 522.0, 14256.0, 468.0],
       ...,
       [12381, '2023-05-31 23:00:00', 27914.2, ..., 987.0, 8375.0, 411.0],
       [12382, '2023-06-01 00:00:00', 25545.6, ..., 915.0, 9323.0, 416.0],
       [12383, '2023-06-01 01:00:00', 23652.2, ..., 914.0, 9454.0, 385.0]],
      dtype=object)